## SBST Core Usage

This Python notebook file explains the example usage of sbst_core.py

1. Import requirements

In [1]:
# Import requirements
from sbst_core import instrument_and_load, FitnessCalculator, hill_climbing_search
import ast
import random

2. Load the basic information of functions and their branches

In [2]:
file_path = "../benchmark/ex5.py" # path to target

source = open(file_path).read()   
namespace, traveler, record, instrumented_tree = instrument_and_load(source)

# Example: pick first function
func_info = traveler.functions[0]
func_name = func_info.name
func_args = func_info.args
func_dims = func_info.args_dim
func_obj = namespace[func_name]
func_max_const, func_min_const = func_info.max_const, func_info.min_const

print(func_info)
print("Max/Min const:", func_max_const, func_min_const)

Function collection with 3 arg(s): ['a', 'b', 'c']
Max/Min const: 1038482 -10


3. List the branches in the function and choose branch

In [3]:
# list branches in the function
branches = traveler.branches.get(func_name, {})
print("Branches:", branches.keys())

# Choose a branch lineno and an outcome to target (True: taken, False: not taken)
target_lineno = next(iter(branches.keys())) # e.g., Choose the first branch
branch_info = branches[2]
target_branch_node = branch_info.node
target_outcome = True   # e.g. try to make branch taken
subject_node = branch_info.subject
parent_map = traveler.parent_map

print(branch_info)

Branches: dict_keys([1, 2, 3])
Branch bid=2 in func=collection


4. Create fitness calculator and calculate fitness for initial random input

In [4]:
# create fitness calculator
fitness_calc = FitnessCalculator(traveler, record, namespace)

# make an initial candidate (random ints for each arg)
initial = [random.randint(-20, 20) for _ in func_args]
initial = [10001]

fitness = fitness_calc.fitness_for_candidate(func_obj, initial, target_branch_node, target_outcome, subject_node, parent_map)
print("initial fitness:", fitness, "for", initial)

initial fitness: 2.0 for [10001]


5-1. (Optional step) Get actual variables and start hill climbing

In [5]:
# get actual indices that matter (args appearing in branch)
actual_vars = {n.id for n in ast.walk(target_branch_node) if isinstance(n, ast.Name)}
actual_indices = [i for i, a in enumerate(func_args) if a in actual_vars]
if not actual_indices:
    actual_indices = list(range(len(func_args)))

# hill-climb
result = hill_climbing_search(func_obj, initial, target_branch_node, target_outcome, fitness_calc, actual_indices, parent_map, subject_node)
print("found:", result)

AttributeError: 'BranchInfo' object has no attribute '_fields'

5-2. ...or attach to simple HC

In [6]:
import itertools
import random

def generate_neighbors(current_inputs: list[int], step_size: int) -> list[list[int]]:
    steps = itertools.product([0, -step_size, step_size], repeat=len(current_inputs))
    neighbors = []

    for step in steps:
        if all(s == 0 for s in step): continue # Skip the inputs themselves
        neighbors.append([input + s for input, s in zip(current_inputs, step)])

    return neighbors

def hill_climb(step: int = 1, max_iter: int = 10000) -> tuple[list[int], float, int]:
    x = [random.randint(func_min_const, func_max_const) for _ in func_args]
    fitness = fitness_calc.fitness_for_candidate(func_obj, x, target_branch_node, target_outcome, subject_node, parent_map)

    for i in range(max_iter):
        neighbors = generate_neighbors(x, step)
        scored = [(n, fitness_calc.fitness_for_candidate(func_obj, n, target_branch_node, target_outcome, subject_node, parent_map)) for n in neighbors]

        best_neighbor, best_neighbor_fitness = min(scored, key=lambda t: t[1])

        if best_neighbor_fitness <= fitness:
            x, fitness = best_neighbor, best_neighbor_fitness

        if fitness == 0:
            break

        if i % 10 == 0:
            print(f'Iteration #{i} best x: {x} (fitness: {fitness})')

    return x, fitness, i

best_x, best_f, iteration_n = hill_climb()
print(f"Found best after {iteration_n} iters. Best:", best_x, "Fitness:", best_f)

Iteration #0 best x: [416782, 924398, 548328] (fitness: 2.0)
Iteration #10 best x: [416782, 924398, 548318] (fitness: 2.0)
Iteration #20 best x: [416782, 924398, 548308] (fitness: 2.0)
Iteration #30 best x: [416782, 924398, 548298] (fitness: 2.0)
Iteration #40 best x: [416782, 924398, 548288] (fitness: 2.0)
Iteration #50 best x: [416782, 924398, 548278] (fitness: 2.0)
Iteration #60 best x: [416782, 924398, 548268] (fitness: 2.0)
Iteration #70 best x: [416782, 924398, 548258] (fitness: 2.0)
Iteration #80 best x: [416782, 924398, 548248] (fitness: 2.0)
Iteration #90 best x: [416782, 924398, 548238] (fitness: 2.0)
Iteration #100 best x: [416782, 924398, 548228] (fitness: 2.0)
Iteration #110 best x: [416782, 924398, 548218] (fitness: 2.0)
Iteration #120 best x: [416782, 924398, 548208] (fitness: 2.0)
Iteration #130 best x: [416782, 924398, 548198] (fitness: 2.0)
Iteration #140 best x: [416782, 924398, 548188] (fitness: 2.0)
Iteration #150 best x: [416782, 924398, 548178] (fitness: 2.0)
Ite